[View in Colaboratory](https://colab.research.google.com/github/NgoDinh/Data-science/blob/master/odoo_bc_sl_size.ipynb)

In [0]:
# -*- coding: utf-8 -*-
##############################################################################
#
#    OpenERP, Open Source Management Solution
#
##############################################################################
import time
import os
import base64
import pandas as pd
import psycopg2 as pg
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine
from openerp.osv import fields, osv
from openerp.tools.translate import _
from openerp.tools import DEFAULT_SERVER_DATE_FORMAT, DEFAULT_SERVER_DATETIME_FORMAT
from dateutil import parser

class report_online_performance(osv.osv_memory):
    
    _name = "report.online.performance"
    _columns = {
        'date_start': fields.date('Date From', required=True),
        'date_end': fields.date('Date To', required=True),
        'file_output':fields.binary(string="File Output",invisible=True, readonly=True, help='Output file in xlsx format'),
        'file_name':fields.char(string='File Name', invisible=True)
    }
    _defaults = {
         'date_start': time.strftime('%Y-%m-01'),
         'date_end': lambda *a: str(datetime.now() + relativedelta(months=+1, day=1, days=-1))[:10],
    }
    
#     def print_report(self, cr, uid, ids, context=None):
#         if context is None:
#             context = {}
#         datas = {'ids': context.get('active_ids', [])}
#         res = self.read(cr, uid, ids, ['date_start', 'date_end'], context=context)
#         res = res and res[0] or {}
#         datas['form'] = res
#         if res.get('id',False):
#             datas['ids']=[res['id']]
#         return {'type': 'ir.actions.report.xml', 'report_name': 'bc_doanh_thu_theo_location' , 'datas': datas}
    
    def print_excel(self, cr, uid, ids, context=None):

       
        #set date to query_data
        #============================================================================================#        
        datas = {'ids': context.get('active_ids', [])}
        date_start = self.read(cr, uid, ids, ['date_start'], context=context)
        date_start = date_start[0]['date_start']+' 00:00:00'
        date_end   = self.read(cr, uid, ids, ['date_end'], context=context)
        date_end   = date_end[0]['date_end']+' 23:59:59'
        out_path = '/tmp/bc_ton_size_%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        engine = pg.connect('postgresql://odoo:odoo@***:***/***')
        #Run report
        #============================================================================================# 
        #get data
        def get_data(cat, target_date):

            #create query to get cat_id
            def get_product_id (cat):
                product_name = {'women': ('DEP NU','GIAY NU','SANDAL NU'), 'men': ('DEP NAM','GIAY NAM'), 'other':('PHU KIEN', 'PHU KIEN')}

                query='''
                    SELECT name,id 
                    FROM product_category 
                    WHERE parent_id in
                        (SELECT id 
                        FROM product_category 
                        WHERE name in %s) %s
                    ORDER BY name
                '''
                if cat == 'kid':
                    query = '''
                        SELECT name,id 
                        FROM product_category 
                        WHERE  (name like '%DEP%' AND parent_id in
                                (SELECT id 
                                FROM product_category 
                                WHERE name in ('TRE EM')))
                            OR 
                              name like 'SANDAL (KSD)'
                            OR
                               parent_id in
                                        (SELECT id 
                                        FROM product_category 
                                        WHERE name in ('GIAY TRE EM'))
                        ORDER BY name
                    '''
                elif cat == 'men':
                    query = query%(product_name['men'], "or name LIKE '%%MSD%%'")
                else:
                    query = query%(product_name[cat], "")
                return query

            #get cat_id
            df_cat = pd.read_sql(get_product_id(cat),engine)
            cat = tuple(df_cat['id'])

            #get data by cat
            get_data_query = '''
            SELECT foo.categ_id, foo.size,sum(foo.qty)::int  as sl
            FROM 
            (SELECT
                quant.id AS id,
                quant.product_id AS product_id,
                quant.location_id AS location_id,
                quant.qty AS qty,
                quant.lot_id AS lot_id,
                quant.package_id AS package_id,
                quant.in_date AS in_date,
                quant.company_id,
                template.categ_id AS categ_id,
                right(prod.default_code,2) as size
            FROM stock_quant AS quant
            JOIN product_product prod ON prod.id = quant.product_id
            JOIN product_template template
                ON template.id = prod.product_tmpl_id) foo
            WHERE foo.location_id in (select id from stock_location where usage = 'internal' and name != 'OUTLET')
                     AND foo.in_date < '%s' and foo.categ_id in %s
            GROUP by 1,2
            ORDER by 1'''%(target_date, str(cat))
            df = pd.read_sql(get_data_query,engine)

            return df, df_cat, cat
        
        #create excel
        writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
        workbook  = writer.book
        worksheet = workbook.add_worksheet()

        header_format = workbook.add_format({'bold': True, 'font_color': 'black', 'bg_color': '#B7DEE8',  'border':1, 'align' :'vcenter'})
        number_format = workbook.add_format({'border':1, 'align': 'center'})
        product_type = {0:'men', 1:'women', 2: 'kid', 3: 'other'}
        big_size = workbook.add_format({'bold': True, 'font_size':15})

        worksheet.write(1, 1, 'BÁO CÁO SỐ LƯỢNG TỒN THEO SIZE', big_size)
        worksheet.write(3, 1, 'Đến ngày', header_format)
        worksheet.write(3, 2, date_start, number_format)

        for y in range(3):

            df, df_cat, cat = get_data(product_type[y], date_start)

            start_row = 6
            start_col = 1 + y*4
            name_format = workbook.add_format({'border':1, 'align': 'center', 'bold':True, 'bg_color': '#%02x%02x%02x' % (118, 170 + y*18, 60)})

            worksheet.write(start_row - 1, start_col, product_type[y].upper(), name_format)
            worksheet.write(start_row - 1, start_col + 1, "SIZE", name_format)
            worksheet.write(start_row - 1, start_col + 2, "STOCK", name_format)
            for cat_id in cat:
                len_id = len(df[df['categ_id'] == cat_id])
                if len_id == 0:
                    worksheet.write(start_row, start_col, df_cat[df_cat["id"] == cat_id]['name'].values[0], header_format)
                    worksheet.merge_range(start_row, start_col + 1,start_row, start_col + 2, "Out of Stock", number_format)
                    start_row = start_row + 1
                else:
                    worksheet.merge_range(start_row, start_col, start_row + len_id -1,start_col, df_cat[df_cat["id"] == cat_id]['name'].values[0], header_format)
                    #worksheet.write(start_row, start_col, df_cat[df_cat["id"] == cat_id]['name'].values[0])
                    for i in range(len_id):
                        worksheet.write(start_row + i, start_col + 1, df[df["categ_id"] == cat_id].iloc[i]['size'], number_format)
                        worksheet.write(start_row + i, start_col + 2, df[df["categ_id"] == cat_id].iloc[i]['sl'], number_format)
                    start_row = start_row + len_id

        worksheet.set_column('B:B', 25)  
        worksheet.set_column('E:E', 1.7)
        worksheet.set_column('F:F', 25)
        worksheet.set_column('I:I', 1.7)
        worksheet.set_column('J:J', 25)
        worksheet.hide_gridlines(2)
        workbook.close()
        
        
        #Encode final file to binary and download it.
        #============================================================================================#
        
        PREVIEW_PATH = out_path
        encoded_string = ""     
        with open(PREVIEW_PATH, "r") as image_file:
            encoded_string = base64.encodestring(image_file.read())#.b64encode
        for order in self.browse(cr, uid, ids[0]):
            order.file_name = '%s.xlsx'%(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            order.file_output = encoded_string
            data_id =  order.id
         
        return {
            'type': 'ir.actions.act_window',
            'res_model': 'report.online.performance',
            'view_mode': 'form',
            'view_type': 'form',
            'res_id': data_id,
            'views': [(False, 'form')],
            'target': 'new',
        }

# vim:expandtab:smartindent:tabstop=4:softtabstop=4:shiftwidth=4:
